In [1]:
import pandas as pd
import numpy as np
import pyodbc 
import time
from datetime import datetime
import sqlite3

In [2]:
import sys
import logging
path = "..\..\InSyncConnection\Code\clinical_log.txt"
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("Primary Care Provider Tracker")

In [3]:
conn = sqlite3.connect('../../InSyncConnection/Database/InSyncClinical.db')
# cursor = conn.cursor()

# # printing all table names  
# sql_query = """SELECT name FROM sqlite_master
#     WHERE type='table';"""

# cursor.execute(sql_query)
# print(cursor.fetchall())

## Read in Data 

### Get new data from sql

In [4]:
# Get current provider data from sql 
try:
    sql='''
    SELECT 
        PatientID,
        PatientPrimaryProviderID AS 'ProviderID'
    FROM 
        tblPatientPrimaryProviders
    WHERE
       IsDefault LIKE 'True'
    '''
    new_provider_df = pd.read_sql(sql, conn)    
    logger.info(f"Successfully queried tblPatientPrimaryProviders.")            
except Exception as e:
    logger.error(f"Failed to query tblPatientPrimaryProviders.", exc_info=True) 
    print(e)
    sys.exit(1)

In [5]:
# Get patient data
try:
    sql='''
    SELECT 
        PatientId AS PatientID,
        FirstName,
        LastName,
        DOB
    FROM 
        emr_PatientDetails
    '''
    patient_details_df = pd.read_sql(sql, conn)
    patient_details_df['FirstName'] = patient_details_df['FirstName'].apply(lambda name: name.capitalize())
    patient_details_df['LastName'] = patient_details_df['LastName'].apply(lambda name: name.capitalize())
    logger.info(f"Successfully queried emr_PatientDetails.")            
except Exception as e:
    logger.error(f"Failed to query emr_PatientDetails.", exc_info=True) 
    print(e)
    sys.exit(1)

In [6]:
# Get previous provider info
useBackup = False
try:
    sql = '''SELECT * FROM ptCurrent_Provider_Tracker'''
    old_provider_df = pd.read_sql(sql, conn)
    logger.info(f"Successfully queried ptCurrent_Provider_Tracker.")
except Exception as e:
    useBackup = True
    logger.error(f"Failed to query ptCurrent_Provider_Tracker.", exc_info=True)
    print(e)

# get previous changes info
try:
    sql = '''SELECT * FROM ptPrimaryProvider_ChangeLog'''
    prev_changes_df = pd.read_sql(sql, conn)
    logger.info(f"Successfully queried ptOld_Provider_Tracker.")            
except Exception as e:
    useBackup = True
    logger.error(f"Failed to query ptOld_Provider_Tracker.", exc_info=True) 
    print(e)

### Get backed up data from excel
    Use if issue with the db

In [7]:
if useBackup == True:
    try:
        old_provider_df = pd.read_excel(r"../../../TestEnvironment/data/Provider Tracker.xlsx",
                                         sheet_name = "Old Provider")
        prev_changes_df = pd.read_excel(r"../../../TestEnvironment/data/Provider Tracker.xlsx",
                                         sheet_name = "Changes")
        logger.info("Successfully read in old data from excel backup")
    except Exception as e:
        logger.error("Failed to read in old data from excel backup")
prev_changes_df

,PatientID,ProviderID,Date Of Change
0,627637,1566,02/07/2024
1,621667,1572,02/07/2024
2,623926,2067,02/07/2024
3,628066,1572,02/07/2024
4,628068,1572,02/07/2024
...,...,...,...
9127,622419,1599,06/14/2023
9128,622423,1599,06/14/2023
9129,622465,1599,06/14/2023
9130,622475,1599,06/14/2023


## Compare new data to old data 

In [8]:
try:    
    #merge frames
    consistant_providers = old_provider_df.merge(new_provider_df,how='outer',on='PatientID', indicator=False)
    
    #filter out consistant data
    inconsistant_mask = consistant_providers['ProviderID_x'] != consistant_providers['ProviderID_y']
    inconsistantProviders = consistant_providers[inconsistant_mask].copy()
    
    # reformat frame
    inconsistantProviders.drop('ProviderID_x', axis = 1, inplace=True)
    inconsistantProviders.rename(columns={'ProviderID_y': 'ProviderID'}, inplace=True)
    
    # add date of change
    today = datetime.today().strftime('%m/%d/%Y')
    inconsistantProviders['Date Of Change'] = today
    
    # concat prev changes
    all_changes = pd.concat([inconsistantProviders,prev_changes_df])
    logger.info(f"Successfully compared old data to new data.")            
except Exception as e:
    logger.error(f"Failed to compare old data to new data.", exc_info=True) 
    print(e)
    sys.exit(1)

## Push data to DB

In [9]:
# ", ".join([item + " " + str(new_provider_df[item].dtype) for item in new_provider_df.columns])

In [10]:
# changes ptCurrent_Provider_Tracker to today's current provider
table_name = "ptCurrent_Provider_Tracker"
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER, ProviderID INTEGER)''')
    conn.commit()
    new_provider_df.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed {table_name} to database.")
    new_provider_df.to_csv()
except Exception as e:
    logger.error(f"Failed to push {table_name} to database.", exc_info=True) 
    print(e)

In [11]:
# ", ".join([item + " " + str(all_changes[item].dtype) for item in all_changes.columns])

In [12]:
# adds the new provider changes to ptPrimaryProvider_ChangeLog
table_name = "ptPrimaryProvider_ChangeLog"
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER, ProviderID INTEGER, DateofChange TEXT)''')
    conn.commit()
    all_changes.to_sql(table_name, conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed {table_name} to database.")            
except Exception as e:
    logger.error(f"Failed to push {table_name} to database.", exc_info=True) 
    print(e)

In [13]:
conn.close()

## Write backup to excel 

In [14]:
# write data to excel as a backup in case something happens to the db
try:
    with pd.ExcelWriter('../../../TestEnvironment/data/Provider Tracker.xlsx') as writer:  
        new_provider_df.to_excel(writer, sheet_name='Old Provider', index = False)
        all_changes.to_excel(writer, sheet_name='Changes', index = False)
    logger.info("Successfully backed up changes to excel.")
except Exception as e:
    logger.error("Failed to back up changes to excel.")
    print(e)